In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
sns.set(font_scale=2.5)

import missingno as msno
from sklearn.preprocessing import LabelEncoder

#ignore warnings 
import warnings
warnings.filterwarnings('ignore')

%matplotlib inlinethe current session

In [ ]:
df = pd.read_csv('../input/advertising/advertising.csv')
df.head()

In [ ]:
for col in df.columns : 
    msg = 'columnn : {:>10}\t count of NaN value : {:.0f}'.format(col, 100 * (df[col].isnull().sum() ))
    print(msg)

In [ ]:
df.info()

In [ ]:
int_column = df.dtypes[df.dtypes =='int64'].index |  df.dtypes[df.dtypes =='float64'].index

In [ ]:
df.describe()

In [ ]:
for col in int_column : 
    plt.figure(figsize=(12,4))
    
    plt.subplot(1,2,1)
    sns.distplot(df[col])
    plt.xlabel(col)
    plt.ylabel('Density')
    
    plt.subplot(1,2,2)
    sns.boxplot(x='Clicked on Ad', y = col, data =df, showmeans = True)
    plt.xlabel('Target')
    plt.ylabel(col)
    
    plt.show()

In [ ]:
obj_column = df.dtypes[df.dtypes == 'object'].index
for i in range(0, len(obj_column)) :
    print(obj_column[i])
    print(len(df[obj_column[i]].unique()))
    print()

In [ ]:
df.head()

In [ ]:
plt.style.use("fivethirtyeight")
sns.jointplot(df["Daily Time Spent on Site"], df.Age, kind='kde')

In [ ]:
sns.pairplot(df, hue='Clicked on Ad')

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(), annot=True)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")
        
    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

In [ ]:
df

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop(['Timestamp', 'Clicked on Ad', 'Ad Topic Line', 'Country', 'City'], axis=1)
y = df['Clicked on Ad']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


num_columns = ['Daily Time Spent on Site', 'Age', 'Area Income', 'Daily Internet Usage', 'Male']


ct = make_column_transformer(
    (MinMaxScaler(), num_columns),
#     (StandardScaler(), num_columns),
    remainder='passthrough'
)

X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)
X_train = pd.DataFrame(X_train, columns=num_columns)
X_test = pd.DataFrame(X_test, columns=num_columns)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train, y_train)

print_score(lr_clf, X_train, y_train, X_test, y_test, train=True)
print_score(lr_clf, X_train, y_train, X_test, y_test, train=False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier



models = [LogisticRegression(),
          RandomForestClassifier(),
          XGBClassifier()
         ]

names = [ 'LogisticRegression',
          'RandomForestClassifier',
          'XGBClassifier'
        ]

for model,name in zip(models,names):
  m = model.fit(X_train,y_train)
  print(name, 'report:')
  print('Train score',model.score(X_train,y_train))
  print('Test score',model.score(X_test,y_test))
  print()
  print("Train confusion matrix:\n",confusion_matrix(y_train, model.predict(X_train)),'\n')
  print("Test confusion matrix:\n",confusion_matrix(y_test, model.predict(X_test)))
  print('*'*50)



In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score

In [ ]:
m = RandomForestClassifier().fit(X_train,y_train)
pred_y = m.predict(X_test)
print('*'*50)
print('Report')
print('model : RandomForestClassifier')
print('Train score',model.score(X_train,y_train))
print('Test score',model.score(X_test,y_test))
print()
print("accuracy: %.2f" %accuracy_score(y_test, pred_y))
print("Precision : %.3f" % precision_score(y_test, pred_y))
print("Recall : %.3f" % recall_score(y_test, pred_y))
print("F1 : %.3f" % f1_score(y_test, pred_y))
print()
print("Train confusion matrix:\n",confusion_matrix(y_train, model.predict(X_train)),'\n')
print("Test confusion matrix:\n",confusion_matrix(y_test, model.predict(X_test)))
print('*'*50)

In [ ]:
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train, y_train)

print('Train score',model.score(X_train,y_train))
print('Test score',model.score(X_test,y_test))

lrCoef = LogisticRegression().fit(X_train,y_train).coef_
print(lrCoef)

In [ ]:
coefdf = pd.DataFrame(data=X_train.columns, index=range(0, len(lrCoef[0])), columns=['Feature'])
coefdf['Coef'] = lrCoef[0]
coefdf['Absuolute num of Coef'] = abs(lrCoef[0])
coefdf = coefdf.sort_values(by='Absuolute num of Coef', ascending=False).reset_index(drop=True)
coefdf

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
sns.barplot(data=coefdf, y=coefdf['Feature'], x=coefdf['Coef'])

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.title('Coefficient of Logistic Regression\n(score : 95%)', fontsize=20)
plt.xlabel('Coefficient')

plt.savefig('Coefficient of Logistic Regression.png')
plt.show()

In [ ]:
import shap

In [ ]:
XGBmodel = XGBClassifier().fit(X_train, y_train)

explainer = shap.Explainer(XGBmodel)
shap_values = explainer(X_train)

# visualize the first prediction's explanation
shap.plots.waterfall(shap_values[0], show=False)
plt.title('SHAP of XGBoost Classifier\n(score : 95%)')
plt.figsize=(10,5)
plt.show()

In [ ]:
RFmodel = RandomForestClassifier().fit(X_train, y_train)

explainer = shap.TreeExplainer(RFmodel)
shap_values = explainer.shap_values(X_test)

# fig, ax = plt.subplots(figsize=(10,5))
shap.summary_plot(shap_values, X_test, plot_size=(10,5), show=False)
plt.title('SHAP of Random Forest Classifier\n(score : 96%)')
plt.show()

In [ ]:
from sklearn.inspection import permutation_importance

result = permutation_importance(RFmodel, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2)
sorted_idx = result.importances_mean.argsort()

plt.figure(figsize=(10,5))
plt.title('Permutation Importance of Random Forest Classifier')

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.boxplot(result.importances[sorted_idx].T,
            vert=False, labels=X.columns[sorted_idx]);

In [ ]:
from pdpbox import pdp, get_dataset, info_plots

# Create the data that we will plot
pdp_goals = pdp.pdp_isolate(model=RFmodel, dataset=X_train,
                            model_features=X_train.columns, feature='Daily Internet Usage')

# plot it
pdp.pdp_plot(pdp_goals, 'Daily Internet Usage')
plt.show()

**Daily Internet Usage** is the most important feature. 

The lower Daily Internet Usage and Daily Time Spent on Site, the more likely to click the ad.

Sex and age are the least relevant feature.

Area Income affect a little.

Thus, targeting Ad to the people who use internet little and rarely spend time on website is efficient to make more likely to click the ad.